<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [ ]:
%load_ext sql

And then we import our movie database

In [ ]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [ ]:
%%sql
PRAGMA foreign_keys=ON;

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [ ]:
%%sql


+ Show the performance dates for one of the movies.

In [ ]:
%%sql


+ Show all data concerning performances at a given theatere
  on a given date.

In [ ]:
%%sql


+ List all customers

In [ ]:
%%sql


+ List all tickets

In [ ]:
%%sql


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [ ]:
%%sql


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2020), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [ ]:
%%sql


+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [ ]:
%%sql


+ Create a ticket where either the user or the performance
  doesn’t exist.

In [ ]:
%%sql
